In [1]:
import numpy as np
import pandas as pd
from scipy.interpolate import CubicSpline
import tkinter as tk
from tkinter import filedialog
import os
import glob

C:\Users\Matthew\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.25.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
root = tk.Tk()
root.lift()
root.withdraw()
#file = filedialog.askopenfilenames(title='Tract File', filetypes=[("Points","*.pts*")])
path = filedialog.askdirectory(title='Fiber Directory')
file = glob.glob(path+'/*.pts')
if not os.path.exists(path+'/Resample_0.5'):
    os.makedirs(path+'/Resample_0.5')

In [3]:
for Filename in file:
    #Filename = File[0].split('/')[-1][:-4]
    Points = np.loadtxt(path +'/'+ Filename.split('\\')[-1][:-4]+".pts")
    Edges = np.loadtxt(path +'/'+ Filename.split('\\')[-1][:-4]+".edge", dtype=int)

    Tracts = np.where(np.diff(Edges[:,0]) > 1)[0]
    #Tracts = [Tracts[0]]
    unitDistance = 0.5

    NewBundle = np.zeros((0,3))
    NewEdge = np.zeros((0,2))

    lastEdge = 0
    startIndex = 0
    for tract in Tracts:
        streamlines = Points[Edges[startIndex][0]:Edges[tract][1]+1,:]

        vectorDistanceX = np.zeros(streamlines.shape[0])
        inclusion = np.ones(vectorDistanceX.shape, dtype=bool)
        for i in range(1,len(vectorDistanceX)):
            vectorize = streamlines[i,:]-streamlines[i-1,:]
            rssqLength = np.sqrt(np.sum(vectorize**2))
            if rssqLength == 0:
                inclusion[i] = False
            vectorDistanceX[i] = rssqLength+vectorDistanceX[i-1]

        f = CubicSpline(vectorDistanceX[inclusion], streamlines[inclusion,:], bc_type='natural')
        newVectorDistance = np.arange(0, vectorDistanceX[-1]+0.1, unitDistance)
        newStreamlines = f(newVectorDistance)

        """
        inclusion = np.ones(newStreamlines.shape[0], dtype=bool)
        for i in range(0, newStreamlines.shape[0]):
            for j in range(i+1, newStreamlines.shape[0]):
                if np.sqrt(np.sum((newStreamlines[i,:]-newStreamlines[j,:])**2)) < 0.4:
                    inclusion[i] = False
        newStreamlines = newStreamlines[inclusion,:]
        """

        NewBundle = np.concatenate((NewBundle, newStreamlines))
        NewEdge = np.concatenate((NewEdge, np.array([np.arange(0,newStreamlines.shape[0]-1)+lastEdge,np.arange(1,newStreamlines.shape[0])+lastEdge]).T))

        startIndex = tract+1
        lastEdge += newStreamlines.shape[0]

    np.savetxt(path+'/Resample_0.5/'+Filename.split('\\')[-1][:-4]+".pts", NewBundle, fmt='%.3f', delimiter=' ')
    np.savetxt(path+'/Resample_0.5/'+Filename.split('\\')[-1][:-4]+".edge", NewEdge, fmt='%d', delimiter=' ')

    #np.savetxt(Filename[0][:-4]+"_single.pts", streamlines, fmt='%.3f', delimiter=' ')
    #np.savetxt(Filename[0][:-4]+"_single.edge", Edges[0:tract+1,:], fmt='%d', delimiter=' ')